In [2]:
                                                          #----------------------------------------
                                                                   #Student Information#
                                                          #----------------------------------------
                                                           # David Engstrom - Student ID: 301537614
                                                           # Brian Salas    - Student ID: 301789398

# This dataset is for our second (student choice) neural network - using MNIST Fashion

#Import Pytorch
import torch                      # Deep learning framework
#Import torch neural network
import torch.nn as nn             # Neural network layers, activations, etc.
#Import torch optimization
import torch.optim as optim       # Optimization algorithms (Adam, SGD, etc.)
#Get the torch DataLoader class
from torch.utils.data import DataLoader  # For batching and shuffling data
#Get the ready-to-use datasets and image preprocessing functions
from torchvision import datasets, transforms # Provides datasets like FashionMNIST
#Handles file paths cleaner and safer across OS
from pathlib import Path

#----------------------------
# Config / Hyperparameters
#----------------------------
BATCH_SIZE = 128   # Number of samples per mini-batch
EPOCHS = 5         # Number of times the dataset is processed fully
LR = 1e-3          # Learning rate for optimizer
SEED = 11          # Random seed for reproducibility
DEVICE = "cpu"     # Run on CPU only

torch.manual_seed(SEED)  # Fix randomness for reproducibility

#----------------------------
# Fashion-MNIST preprocessing
#----------------------------
transform = transforms.Compose([
    transforms.ToTensor(),               # Convert images to tensors
    transforms.Normalize((0.5,), (0.5,)) # Normalize pixel values to mean=0.5, std=0.5
])

#Load the Fashion-MNIST training dataset (60,000 images)
train_ds = datasets.FashionMNIST(root="data", train=True, download=True, transform=transform)

#Load the Fashion-MNIST test dataset (10,000 images)
test_ds  = datasets.FashionMNIST(root="data", train=False, download=True, transform=transform)

#Wrap training dataset in DataLoader: batches of BATCH_SIZE, shuffled each epoch
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

#Wrap test dataset in DataLoader: batches of BATCH_SIZE, no shuffle
test_loader  = DataLoader(test_ds,  batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

#----------------------------
# Define a Convolutional Neural Network (CNN) for Fashion-MNIST
#----------------------------
class FashionCNN(nn.Module):
    def __init__(self):
        super().__init__()

        # Convolutional feature extractor
        self.features = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),   # input=1 channel (grayscale), output=32 filters
            nn.ReLU(inplace=True),                        # activation function
            nn.Conv2d(32, 64, kernel_size=3, padding=1),  # 32 → 64 filters
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),                              # downsample: 28x28 → 14x14

            nn.Conv2d(64, 128, kernel_size=3, padding=1), # 64 → 128 filters
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),                              # downsample: 14x14 → 7x7
        )

        # Fully connected classifier
        self.classifier = nn.Sequential(
            nn.Flatten(),                                 # flatten into 1D vector
            nn.Linear(128 * 7 * 7, 256),                  # fully connected layer
            nn.ReLU(inplace=True),                        # activation
            nn.Dropout(0.3),                              # dropout to reduce overfitting
            nn.Linear(256, 10)                            # output layer: 10 classes (clothing categories)
        )

    def forward(self, x):
        x = self.features(x)      # apply convolution + pooling
        x = self.classifier(x)    # apply fully connected layers
        return x                  # return logits (raw class scores)

# Initialize the model
model = FashionCNN().to(DEVICE)

#----------------------------
# Loss & Optimizer
#----------------------------
criterion = nn.CrossEntropyLoss()                 # Cross entropy for multi-class classification
optimizer = optim.Adam(model.parameters(), lr=LR) # Adam optimizer

# ---------------------------
# Train loop for one epoch
# ---------------------------
def train_one_epoch(epoch):
    model.train()                            # training mode
    running_loss, correct, total = 0.0, 0, 0 # track stats

    for images, labels in train_loader:      # loop over training data
        images, labels = images.to(DEVICE), labels.to(DEVICE) # move data to device

        optimizer.zero_grad()                # reset gradients
        logits = model(images)               # forward pass
        loss = criterion(logits, labels)     # compute loss
        loss.backward()                      # backward pass
        optimizer.step()                     # update weights

        running_loss += loss.item() * images.size(0)  # accumulate loss
        preds = logits.argmax(dim=1)                  # predicted classes
        correct += (preds == labels).sum().item()     # correct predictions
        total += labels.size(0)                       # total samples

    epoch_loss = running_loss / total                 # average loss
    epoch_acc = correct / total                       # accuracy
    print(f"Epoch {epoch}: train loss={epoch_loss:.4f}, acc={epoch_acc:.4f}")

# ---------------------------
# Evaluation function
# ---------------------------
@torch.no_grad()                   # no gradient tracking
def evaluate():
    model.eval()                   # evaluation mode
    correct, total = 0, 0

    for images, labels in test_loader:  # loop over test data
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        logits = model(images)          # forward pass
        preds = logits.argmax(dim=1)    # predictions
        correct += (preds == labels).sum().item()
        total   += labels.size(0)

    return correct / total              # return accuracy

# ---------------------------
# Run training loop
# ---------------------------
best_acc = 0.0                                     # best accuracy so far
Path("checkpoints").mkdir(exist_ok=True)           # make checkpoints folder

for epoch in range(1, EPOCHS + 1):                 # for each epoch
    train_one_epoch(epoch)                         # train model
    test_acc = evaluate()                          # evaluate model
    print(f"Test acc after epoch {epoch}: {test_acc:.4f} ({test_acc*100:.2f}%)")

    if test_acc > best_acc:                        # save if accuracy improves
        best_acc = test_acc
        torch.save(model.state_dict(), "checkpoints/fashion_cnn_best.pt")
        print(f"Saved new best model with acc={best_acc:.4f} ({best_acc*100:.2f}%)")

print(f"Best test accuracy: {best_acc:.4f} ({best_acc*100:.2f}%)")


Epoch 1: train loss=0.4502, acc=0.8360
Test acc after epoch 1: 0.8799 (87.99%)
Saved new best model with acc=0.8799 (87.99%)
Epoch 2: train loss=0.2632, acc=0.9055
Test acc after epoch 2: 0.9082 (90.82%)
Saved new best model with acc=0.9082 (90.82%)
Epoch 3: train loss=0.2152, acc=0.9224
Test acc after epoch 3: 0.9190 (91.90%)
Saved new best model with acc=0.9190 (91.90%)
Epoch 4: train loss=0.1811, acc=0.9338
Test acc after epoch 4: 0.9238 (92.38%)
Saved new best model with acc=0.9238 (92.38%)
Epoch 5: train loss=0.1551, acc=0.9433
Test acc after epoch 5: 0.9220 (92.20%)
Best test accuracy: 0.9238 (92.38%)
